# MAGIC Gamma Telescope

In this notebook, we will load data simulating the registration of high-energy gamma particles in an atmospheric Cherenkov telescope. <a href="https://archive.ics.uci.edu/dataset/159/magic+gamma+telescope" target="_blank" rel="noopener">Follow this link</a> to get details about this dataset.

To execute queries and upload data to the Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the sandbox](../sandbox_config.ipynb).

## Setup

### Access configuration

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

## Download data

First, we will load the data into Pandas DataFrame. Each data column represents one of the features and is named accordingly, see section Additional Variable Information in the dataset description. We will name the Pandas columns as per the variable description.

In [ ]:
from urllib.request import urlopen
import tempfile
from zipfile import ZipFile
from contextlib import ExitStack
import pandas as pd
from stopwatch import Stopwatch

stopwatch = Stopwatch()

DATA_URL = "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip"
DATA_FILE = "magic04.data"

resp = urlopen(DATA_URL)
with ExitStack() as stack:
    f = stack.enter_context(tempfile.TemporaryFile())
    f.write(resp.read())
    print(f"Downloading the data took: {stopwatch}")

    f.seek(0)
    z = stack.enter_context(ZipFile(f))
    f = stack.enter_context(z.open(DATA_FILE, "r"))
    df = pd.read_csv(f)

column_names = [
    'fLength',   # major axis of ellipse [mm]
    'fWidth',    # minor axis of ellipse [mm] 
    'fSize',     # 10-log of the sum of the content of all pixels [in #phot]
    'fConc',     # ratio of the sum of two highest pixels over fSize  [ratio]
    'fConc1',    # ratio of highest pixel over fSize  [ratio]
    'fAsym',     # distance from the highest pixel to center, projected onto major axis [mm]
    'fM3Long',   # 3rd root of the third moment along major axis  [mm] 
    'fM3Trans',  # 3rd root of the third moment along minor axis  [mm]
    'fAlpha',    # angle of major axis with vector to origin [deg]
    'fDist',     # distance from the origin to the center of ellipse [mm]
    'class'      # g,h - gamma (signal), hadron (background)
]
df.columns = column_names

print(df.head())

## Upload data into DB

Let's split data randomly into train and test sets. We will then create two tables - TELESCOPE_TRAIN and TELESCOPE_TEST - and load the datasets into these tables.

In [ ]:
from sklearn.model_selection import train_test_split
from exasol.connections import open_pyexasol_connection

# Split the data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2)

train_table = 'TELESCOPE_TRAIN'
test_table = 'TELESCOPE_TEST'
column_desc = [f'{c} {("DECIMAL(18,4)" if c.startswith("f") else "CHAR(1)")}' for c in column_names]

stopwatch = Stopwatch()

# Create an Exasol connection
with open_pyexasol_connection(sb_config, compression=True) as conn:

    # Create tables
    sql = f'CREATE OR REPLACE TABLE "{sb_config.SCHEMA}"."{train_table}"({", ".join(column_desc)})'
    conn.execute(query=sql)
    sql = f'CREATE OR REPLACE TABLE "{sb_config.SCHEMA}"."{test_table}" LIKE "{sb_config.SCHEMA}"."{train_table}"'
    conn.execute(query=sql)

    # Import data into Exasol
    conn.import_from_pandas(df_train, (sb_config.SCHEMA, train_table))
    print(f"Imported {conn.last_statement().rowcount()} rows into {train_table}.")
    conn.import_from_pandas(df_test, (sb_config.SCHEMA, test_table))
    print(f"Imported {conn.last_statement().rowcount()} rows into {test_table}.")

print(f"Importing the data took: {stopwatch}")